## Step 1.  Authenticate GEE , install and required packeges

In [5]:
# Set up the current working directory
import os

In [6]:
os.getcwd()

'C:\\Users\\LEBELA TADELE MELESE'

In [7]:
os.mkdir('PDSI_predicition')

In [12]:
os.chdir('C:\\Users\\LEBELA TADELE MELESE\\PDSI_predicition')

In [13]:
os.getcwd()

'C:\\Users\\LEBELA TADELE MELESE\\PDSI_predicition'

In [1]:
import ee
import geemap
ee.Authenticate()

True

In [ ]:
!pip install pandas numpy matplotlib geopandas rasterio scikit-learn

## Step 2. Preprocess and export climate variables (from Terraclimate) for Ethiopia

In [2]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

# Define the area of interest (Ethiopia boundary).
ethiopia = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Ethiopia'))

# Load TerraClimate dataset.
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")

# Filter data for the years of interest (e.g., 1980-2023) and Ethiopia
start_date = '1980-01-01'
end_date = '2023-12-31'

# Define the bands of interest
bands = ['pdsi', 'pr', 'tmmx', 'tmmn', 'aet', 'pet', 'soil', 'srad', 'vpd']

# Filter and select bands
terraclimate_filtered = terraclimate.filterDate(start_date, end_date).select(bands)

# Clip to Ethiopia boundary
terraclimate_ethiopia = terraclimate_filtered.map(lambda image: image.clip(ethiopia))

# Define the scale factors for each variable
scale_factors = {
    'aet': 0.1,
    'def': 0.1,
    'pdsi': 0.01,
    'pet': 0.1,
    'pr': 0,
    'ro': 0.1,
    'soil': 0.1,
    'srad': 0.1,
    'swe': 0.1,
    'tmmn': 0.1,
    'tmmx': 0.1,
    'vap': 0.001,
    'vpd': 0.01,
    'vs': 1  # No scale factor provided
}

def apply_scale(image):
    # Apply scale factors to each band
    scaled_image = image
    for band in bands:
        if band in scale_factors:
            # Apply scale factor to the band
            scaled_image = scaled_image.addBands(image.select(band).multiply(scale_factors[band]).rename(band))
    return scaled_image

# Apply scale factors to the image collection
scaled_terraclimate_ethiopia = terraclimate_ethiopia.map(apply_scale)

# Aggregate monthly data
monthly_terraclimate_ethiopia = scaled_terraclimate_ethiopia.mean()

# Verify band names in the aggregated image
def print_band_names(image):
    bands = image.bandNames().getInfo()
    print("Bands in the image:", bands)

# Print band names to debug
print_band_names(monthly_terraclimate_ethiopia)

# Export each variable to Google Drive
for band in bands:
    # Select the band of interest
    if band in monthly_terraclimate_ethiopia.bandNames().getInfo():
        image = monthly_terraclimate_ethiopia.select(band)
        
        # Export the data to Google Drive
        export_task = ee.batch.Export.image.toDrive(
            image=image,
            description=f'Ethiopia_{band}_Monthly_Scaled',
            folder='EarthEngine',
            scale=4000,
            region=ethiopia.geometry(),
            fileFormat='GeoTIFF'
        )
        
        # Start the export task
        export_task.start()
    else:
        print(f"Band {band} not found in the aggregated image.")

print("Export tasks have been started.")


Bands in the image: ['pdsi', 'pr', 'tmmx', 'tmmn', 'aet', 'pet', 'soil', 'srad', 'vpd', 'pdsi_1', 'pr_1', 'tmmx_1', 'tmmn_1', 'aet_1', 'pet_1', 'soil_1', 'srad_1', 'vpd_1']
Export tasks have been started.
